In [6]:
import os
import json
import numpy as np
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
import glob
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, ConcatDataset, Subset

# ==========================================
# 1. Setup
# ==========================================
WANDB_API_KEY = "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"
PROJECT_NAME = "cifar10_mlops_project"
ENTITY = "esi-sba-dz"
wandb.login(key=WANDB_API_KEY)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: WARNING [wandb.login()] Changing session credentials to explicit value for https://api.wandb.ai.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [7]:
# ==========================================
# 2. Helpers (Strict No-Download)
# ==========================================
class Cifar10DataManager:
    def __init__(self, data_dir="./data"):
        self.data_dir = data_dir
        self.mean = (0.4914, 0.4822, 0.4465)
        self.std = (0.2023, 0.1994, 0.2010)

    def get_loader_for_retrain(self, batch_size, architecture_option='standard'):
        tf_list = [transforms.ToTensor(), transforms.Normalize(self.mean, self.std)]
        train_tf = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.RandomCrop(32, 4)] + tf_list)
        if architecture_option == 'upsample':
            # Fix: Ensure ToTensor and Normalize are included for upsample too
            train_tf = transforms.Compose([transforms.Resize(224), transforms.RandomHorizontalFlip()] + tf_list)

        # STRICT: download=False
        train_set = torchvision.datasets.CIFAR10(root=self.data_dir, train=True, download=False, transform=train_tf)
        return train_set, train_tf

    def get_test_loader(self, batch_size, architecture_option='standard'):
        tf_list = [transforms.ToTensor(), transforms.Normalize(self.mean, self.std)]
        test_tf = transforms.Compose(tf_list)
        if architecture_option == 'upsample':
             test_tf = transforms.Compose([transforms.Resize(224)] + tf_list)

        test_set = torchvision.datasets.CIFAR10(root=self.data_dir, train=False, download=False, transform=test_tf)
        indices_path = os.path.join(self.data_dir, "processed", "test_indices.npy")
        if os.path.exists(indices_path):
            test_indices = np.load(indices_path)
            test_set = Subset(test_set, test_indices)

        return DataLoader(test_set, batch_size=batch_size, shuffle=False)

    def get_simulation_raw(self):
        return torchvision.datasets.CIFAR10(root=self.data_dir, train=False, download=False)

def build_model(architecture_option='standard'):
    model = torchvision.models.resnet18(pretrained=True)
    if architecture_option == 'modified':
        model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        model.maxpool = nn.Identity()
    elif architecture_option == 'upsample':
        pass
    model.fc = nn.Linear(model.fc.in_features, 10)
    return model

def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        loss = criterion(model(inputs), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(loader)

def validate(model, loader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            running_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return running_loss / len(loader), 100 * correct / total

In [8]:
# ==========================================
# 3. Execution
# ==========================================
run = wandb.init(project=PROJECT_NAME, entity=ENTITY, job_type="retrain", tags=["retrain"])

try:
    print("Downloading Feedback...")
    f_art = run.use_artifact(f'{ENTITY}/{PROJECT_NAME}/cifar10-feedback:latest').download(root=".")
    feedback = np.load(os.path.join(f_art, "feedback_v1.npy"))
except:
    print("No feedback found.")
    feedback = []

if len(feedback) > 0:
    print("Downloading Baseline Data...")
    run.use_artifact(f'{ENTITY}/{PROJECT_NAME}/cifar10_dataset:latest').download("./data")

    api = wandb.Api()
    sweeps = api.project(PROJECT_NAME, entity=ENTITY).sweeps()
    best_run = api.sweep(f"{ENTITY}/{PROJECT_NAME}/{sweeps[0].id}").best_run()
    config = best_run.config

    print("Downloading Baseline Model...")
    m_dir = best_run.logged_artifacts()[0].download(root="./models")
    m_path = glob.glob(os.path.join(m_dir, "*.pth"))[0]

    # Dataset Merge
    dm = Cifar10DataManager()
    base_train, tf = dm.get_loader_for_retrain(config['batch_size'], config['architecture_option'])
    raw_sim = dm.get_simulation_raw()

    # NEW: Get validation loader
    test_loader = dm.get_test_loader(config['batch_size'], config['architecture_option'])

    class FeedbackDS(torch.utils.data.Dataset):
        def __init__(self, raw, inds, tf):
             self.raw = raw; self.inds = [int(i[0]) for i in inds]; self.tf = tf
        def __len__(self): return len(self.inds)
        def __getitem__(self, i):
             img, label = self.raw[self.inds[i]]
             return self.tf(img), label

    fb_ds = FeedbackDS(raw_sim, feedback, tf)
    loader = DataLoader(ConcatDataset([base_train, fb_ds]), batch_size=config['batch_size'], shuffle=True)

    # Retrain
    model = build_model(config['architecture_option']).to(device)
    model.load_state_dict(torch.load(m_path, map_location=device))

    # NEW: Use config for optimizer to match original training
    lr = config.get('learning_rate', 0.001)
    if config.get('optimizer') == 'sgd':
        opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    else:
        opt = optim.Adam(model.parameters(), lr=lr)

    crit = nn.CrossEntropyLoss()

    print("Fine-tuning (Exact Replication Mode)...")
    for e in range(config['epochs']):
        train_loss = train_epoch(model, loader, crit, opt, device)
        val_loss, val_acc = validate(model, test_loader, crit, device)

        print(f"Epoch {e+1} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
        wandb.log({
            "retrain_loss": train_loss,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "epoch": e
        })

    torch.save(model.state_dict(), "retrained.pth")
    art = wandb.Artifact("retrained-model", type="model")
    art.add_file("retrained.pth")
    run.log_artifact(art)
    print("Retraining Complete.")
else:
    print("Skipping.")

run.finish()

wandb:   1 of 1 files downloaded.  


wandb: Downloading large artifact 'cifar10_dataset:latest', 340.26MB. 11 files...
wandb:   11 of 11 files downloaded.  
Done. 00:00:00.2 (1804.6MB/s)
wandb: Sorting runs by -summary_metrics.val_acc


wandb:   1 of 1 files downloaded.  


Fine-tuning (Exact Replication Mode)...
Epoch 1 | Train Loss: 0.3342 | Val Loss: 0.2943 | Val Acc: 89.72%
Epoch 2 | Train Loss: 0.2410 | Val Loss: 0.2655 | Val Acc: 90.88%
Epoch 3 | Train Loss: 0.1896 | Val Loss: 0.2334 | Val Acc: 91.99%
Epoch 4 | Train Loss: 0.1586 | Val Loss: 0.2177 | Val Acc: 92.54%
Epoch 5 | Train Loss: 0.1246 | Val Loss: 0.2107 | Val Acc: 93.11%
Epoch 6 | Train Loss: 0.1084 | Val Loss: 0.2021 | Val Acc: 93.00%
Epoch 7 | Train Loss: 0.0903 | Val Loss: 0.2049 | Val Acc: 93.26%
Epoch 8 | Train Loss: 0.0763 | Val Loss: 0.2096 | Val Acc: 93.54%
Retraining Complete.


epoch,▁▂▃▄▅▆▇█
retrain_loss,█▅▄▃▂▂▁▁
val_acc,▁▃▅▆▇▇▇█
val_loss,█▆▃▂▂▁▁▂
epoch,7
retrain_loss,0.07632
val_acc,93.5375
val_loss,0.20965
